In [38]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv('../data/process_v1/process_data_4.csv')

In [40]:
del df['unitPrice']

In [41]:
del df['landType']

In [42]:
df.head(3)

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,numberOfKitchens,numberOfGarages,certificateOfLandUseRight,ward,street,district,city,...,lon,typeOfRealEstate,frontWidth,endWidth,facade,houseDirection,landSize,price,accessibility,description
0,1,NaN,NaN,NaN,NaN,no,hoài đức - vân canh,hoài đức - vân canh - hậu ái,hoài đức,hà nội,...,105.727235,privateLand,4.2,NaN,oneSideOpen,southeast,50.0,4.6,fitThreeCars,"bán đất dịch vụ 6,9ha vân canh, hoài đức, hà n..."
1,5,5.0,4.0,NaN,NaN,yes,hoàng mai - giáp bát,hoàng mai - giáp bát - giáp bát,hoàng mai,hà nội,...,105.843605,privateProperty,0.0,NaN,oneSideOpen,NaN,35.0,3.6,fitOneCarAndOneMotorbike,"+ cần bán nhà phố giáp bát, giáp tuyến phố kim..."
2,4,NaN,NaN,NaN,NaN,yes,hà đông - văn quán,hà đông - văn quán - chiến thắng,hà đông,hà nội,...,105.796775,privateProperty,4.5,4.5,oneSideOpen,NaN,40.0,3.9,notInTheAlley,"bán nhà giá bất ngờ 3.9 tỷ,có dt chính 40 m2 ,..."


In [43]:
df['w'] = df['frontWidth']
df['h'] = df['landSize'] / df['w']

In [44]:
df = df.replace(np.inf, np.nan)

In [46]:
df

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,numberOfKitchens,numberOfGarages,certificateOfLandUseRight,ward,street,district,city,...,frontWidth,endWidth,facade,houseDirection,landSize,price,accessibility,description,w,h
0,1,NaN,NaN,NaN,NaN,no,hoài đức - vân canh,hoài đức - vân canh - hậu ái,hoài đức,hà nội,...,4.2,NaN,oneSideOpen,southeast,50.0,4.60,fitThreeCars,"bán đất dịch vụ 6,9ha vân canh, hoài đức, hà n...",4.2,11.904762
1,5,5.0,4.0,NaN,NaN,yes,hoàng mai - giáp bát,hoàng mai - giáp bát - giáp bát,hoàng mai,hà nội,...,0.0,NaN,oneSideOpen,NaN,35.0,3.60,fitOneCarAndOneMotorbike,"+ cần bán nhà phố giáp bát, giáp tuyến phố kim...",0.0,NaN
2,4,NaN,NaN,NaN,NaN,yes,hà đông - văn quán,hà đông - văn quán - chiến thắng,hà đông,hà nội,...,4.5,4.5,oneSideOpen,NaN,40.0,3.90,notInTheAlley,"bán nhà giá bất ngờ 3.9 tỷ,có dt chính 40 m2 ,...",4.5,8.888889
3,7,7.0,7.0,NaN,NaN,yes,hoàn kiếm - lý thái tổ,hoàn kiếm - lý thái tổ - lý thái tổ,hoàn kiếm,hà nội,...,7.0,NaN,twoSideOpen,NaN,120.0,165.00,notInTheAlley,cần bán tòa văn phòng kết hợp phòng ở tầng trê...,7.0,17.142857
4,4,NaN,NaN,NaN,NaN,yes,cầu giấy - nghĩa đô,cầu giấy - nghĩa đô - hoàng quốc việt,cầu giấy,hà nội,...,0.0,NaN,twoSideOpen,NaN,50.0,26.70,notInTheAlley,"+ nhà 4 tầng, kinh doanh hàng ăn, phố kinh doa...",0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170959,1,2.0,2.0,NaN,NaN,no,hai bà trưng - vĩnh tuy,hai bà trưng - vĩnh tuy - mạc thị bưởi,hai bà trưng,hà nội,...,0.0,NaN,oneSideOpen,NaN,70.0,2.70,notInTheAlley,chủ nhà cần bán căn hộ với thông tin chính xác...,0.0,NaN
170960,6,10.0,10.0,NaN,NaN,no,thanh xuân - thượng đình,thanh xuân - thượng đình - thượng đình,thanh xuân,hà nội,...,4.5,NaN,oneSideOpen,NaN,45.0,6.99,narrorRoad,bán nhà đường khương đình thanh xuân kinh doan...,4.5,10.000000
170961,8,22.0,22.0,NaN,NaN,yes,long biên - long biên,long biên - long biên - cổ linh,long biên,hà nội,...,4.3,NaN,twoSideOpen,southwest,82.0,25.00,fitTwoCars,anh chị nhà phố cổ cần bán tòa khách sạn doanh...,4.3,19.069767
170962,5,NaN,NaN,NaN,NaN,yes,nam từ liêm - mỹ đình 1,nam từ liêm - mỹ đình 1 - đình thôn,nam từ liêm,hà nội,...,0.0,NaN,oneSideOpen,NaN,69.0,10.90,notInTheAlley,"cần ngay vốn, bán nhà vị trí mặt tiền tọa lạc ...",0.0,NaN
